我们首先关注具有单隐藏层的多层感知机

In [3]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.randn(2, 4)
net(X)

tensor([[-0.2356],
        [-0.4555]], grad_fn=<AddmmBackward>)

参数访问

In [6]:
print(net[0].state_dict())

OrderedDict([('weight', tensor([[ 0.3484, -0.0829,  0.3569, -0.0451],
        [ 0.2180,  0.4557,  0.3389,  0.1343],
        [ 0.3113, -0.4504,  0.4282, -0.3315],
        [-0.1510,  0.0175, -0.0024, -0.1719],
        [-0.2592, -0.1366,  0.4256, -0.2587],
        [ 0.3610, -0.3211,  0.0144,  0.0845],
        [-0.3418, -0.2644,  0.4435,  0.3301],
        [-0.1721,  0.2693,  0.4576, -0.1684]])), ('bias', tensor([-0.3290,  0.0362, -0.0985,  0.3913,  0.3813,  0.4838, -0.3230,  0.2700]))])


目标参数

In [10]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)
print(net[2].bias.grad)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.2776], requires_grad=True)
tensor([-0.2776])
None


In [12]:
net[2].weight.grad == None # 还没有做运算，所以梯度是None

True

一次性访问所有参数

In [16]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [17]:
net.state_dict()['2.bias'].data

tensor([-0.2776])

从嵌套块收集参数

In [18]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 4),
                         nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[-0.3947],
        [-0.3937]], grad_fn=<AddmmBackward>)

我们已经设计了网络，让我们看看它是如何组织的

In [19]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


内置初始化

In [29]:
def init_normal(m): # 使用正态分布初始化
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)

net.apply(init_normal) # 调用init_normal()
net[0].weight.data[0], net[0].bias.data[0]

(tensor([ 0.0098,  0.0132,  0.0019, -0.0111]), tensor(0.))

In [30]:
def init_constant(m): # 使用特定的值初始化
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)

net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

对某些块应用不同的初始化方法

In [33]:
def xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight) # 均匀初始化

def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

net[0].apply(xavier) # 对0层参数初始化
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([-0.2896, -0.1754, -0.4600, -0.3368])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


自定义初始化

In [37]:
def my_init(m):
    if type(m) == nn.Linear:
        print(
            "Init",
            *[(name, param.shape) for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10) # 均匀分布
        m.weight.data *= m.weight.data.abs() >= 5 # 自身*自身的绝对值若>5则成立，否则置为0

net.apply(my_init) # 调用初始化函数用
net[0].weight[:2] # 访问第0层的参数的第0维度的前两个数值

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[ 0.0000,  0.0000,  6.5338, -9.9987],
        [ 0.0000, -8.5814, -0.0000,  7.8714]], grad_fn=<SliceBackward>)

直接暴力初始化：

In [38]:
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000,  1.0000,  7.5338, -8.9987])

参数绑定

In [39]:
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), shared, nn.ReLU(), shared,
                    nn.ReLU(), nn.Linear(8, 1)) # 两个shared线性层 的 参数是一样的，都指向一个类的实例，下面是验证

net(X)
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


tensor([-2., -1.,  0.,  1.,  2.])